In [124]:
from common import *
from pathlib import Path
import json

In [125]:
class staticclass(type):
    class InstantiationError(Exception):
        pass
    def __init__(cls, name, parents, attrs):
        for k, v in dict.items(attrs):
            if callable(v):
                attrs[k] = staticmethod(v)
    def __call__(cls, *a, **k):
        raise __class__.InstantiationError('cannot instantiate a staticclass')

In [126]:
I = lambda x: x
logavg = lambda x,y: math.expm1((math.log1p(x)+math.log1p(y))/2)
keepSigFig = lambda n: lambda x: round(x, -int(math.floor(math.log10(abs(x)))) + (n - 1)) if x else x

In [127]:
class PSD(O()):
    class Discrete(O()):
        raw = {
            ('max_depth','num_leaves'): #10
                [(6,1<<6),(9,1<<7),(9,1<<9),(12,1<<8),(12,1<<10),(12,1<<12),(-1,1<<8),(-1,1<<10),(-1,1<<12),(-1,1<<14)],
            #('max','num'): #10
            #    [(6,6),(9,7),(9,9),(12,8),(12,10),(12,12),(-1,8),(-1,10),(-1,12),(-1,14)],
        }
        
    class OneByOne(O()):
        class info(O()):
            a = "main [a]rray data of 3 values, [min mid max]"
            b =  "[b]ack up value, i.e. default value if array doesn't give better results"
            cast = "function to apply to values before using to cast to the right dtype"
            lim = "maximum number of iterations of searching in this hyperparameter"
        class default(O()):
            cast = keepSigFig(2)
            lim = 2
        data = {
            'min_data_in_leaf': O(a=[1,60,375], cast=round),
            'min_sum_hessian_in_leaf': O(a=[0,50,200]),
            'lambda_l1': O(a=[0,.02,.2], b=0, lim=1),
            'lambda_l2': O(a=[0,.02,.2], b=0, lim=1),
        }

class ParamSearch():
    mix = staticmethod(logavg)
    
    def __init__(self, psd):
        self.data = O.mycopy(psd)
        self.setup_data()
        
    def setup_data(self):
        dsc = self.data.Discrete
        dsc.keys = list(flatten( dsc.raw.keys() ))
        dsc.assigns = [list(flatten(x)) for x in product(* dsc.raw.values() )]
        obo = self.data.OneByOne
        for k,v in dict.items(obo.data):
            for i,x in dict.items(obo.default):
                if i not in v:
                    v[i] = x
    
    def search(self):
        dsc, obo = self.data.Discrete, self.data.OneByOne
        for assign in dsc.assigns:
            params = dict(zip(dsc.keys, assign))
            coroutine = self.one_by_one()
            for addon in coroutine:
                params.update(addon)
                coroutine.send((yield params)); assert (yield) == None
        
        
    def one_by_one(self):
        obod = self.data.OneByOne.data
        
        #! main algorithm ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        
        # initialize loop variables
        params = {k: (v.b if 'b' in v else v.a[1]) for k,v in dict.items(obod)}
        ranges = {k: v.a for k,v in dict.items(obod)}
        scores = {k: [-np.inf, -np.inf] for k in params}
        isdone = {k: False for k in params}
        
        # pre loop one-off work
        base_score = yield params; assert (yield) == None
        
        # loop
        for i in range(9999999999):
            #! try new parameter values ############################## part A of loop work
            
            # initialize local loop variable
            new_scores = {k: [-np.inf, -np.inf] for k in params}
            new_params = {k: ( obod[k].cast(self.mix(v[0], v[1])),
                               obod[k].cast(self.mix(v[1], v[2])) ) for k,v in dict.items(ranges)}
            
            # finish condition check
            isdone = {k: v or i>=obod[k].lim for k,v in dict.items(isdone)}
            if all(isdone.values()):
                break
                
            # try new parameter values for all parameters
            for key in list(params):
                if i >= obod[key].lim:
                    continue
                orig = params[key]
                params[key] = new_params[key][0]
                scores[key][0] = yield params; assert (yield) == None
                params[key] = new_params[key][1]
                scores[key][1] = yield params; assert (yield) == None
                params[key] = orig
            
            #! start setting up values for next loop ######################## part B of loop work
            
            # set params to the best found and see if it betters score, updating ranges also
            #num_nochange = 0
            for key in list(params):
                if scores[key][0] > base_score and scores[key][0] >= scores[key][1]:
                    params[key] = new_params[key][0]
                    ranges[key] = [ranges[key][0], params[key], ranges[key][1]]
                elif scores[key][1] > base_score and scores[key][1] >= scores[key][0]:
                    params[key] = new_params[key][1]
                    ranges[key] = [ranges[key][1], params[key], ranges[key][2]]
                else:
                    ranges[key] = [new_params[key][0], ranges[key][1], new_params[key][1]]
                    #num_nochange += 1
                
            # send out new params
            #if num_nochange < len(params):
            base_score = yield params; assert (yield) == None

In [10]:
class MSD(O()):
    class Features(O()):
        '''features selection groups'''
        data = [
            ('f1','f2','f3'),
            ('f4','f5','f6')
        ]
    
    class Samples(O()):
        '''sample learning/cv split'''
        code = [
            O(method='sklearn', key=44)
        ]
        
    class Params(O()):
        '''parameters constant settings'''
        data = dict(
            objective = 'binary',
            num_iterations = 10000,
            early_stopping_round = 50,
            learning_rate = .05,
            seed = 44,
            bagging_seed = 45,
            feature_fraction_seed = 46,
        )

class ModelSearch():
    def __init__(self, msd):
        # need:
        self.data = msd
    def what():
        TODO

In [113]:
class IndexFileSystem():
    def __init__(self, directory, key):
        self.dir = directory if isinstance(directory, Path) else Path(directory)
        TODO # if not exists self.dir make it
        assert isinstance(key, str), "key must be string"
        self.key = key
        self.io = O()

    def iterIndices(self):
        for dot_params in self.dir.glob('*.' + self.key):
            yield int(dot_params.stem)

    def getFilePath(self, *, i):
        return self.dir / (str(i) + '.' + name)

    @staticmethod
    def readWrapper(read):
        '''wraps io read operations to safely return None if file does not exist'''
        @wraps(read)
        def read_safely(*a, **k):
            try:
                return read(*a, **k)
            except FileNotFoundError:
                return None
        return read_safely

    def assignIO(self, name, *, read, write, format):
        assert format in ['bytes', 'text'], "argument `format` must be one of 'bytes' or 'text'"
        self.io[name] = O(
            read = readWrapper( lambda *,i: read(getattr(self.getFilePath(name, i=i), 'read_'+format)()) ),
            write = lambda x,*,i: getattr(self.getFilePath(name, i=i), 'write_'+format)(write(x))
        )

class IndexDataStore():    
    def __init__(self, file):
        self.file = file
        self.op = O()
        self.lists = O()
        self.key = None
        self.keyFunc = lambda keyVal, *, client=False: object()
        self.tbl = {}
        self.nextIndex = 0
        
    def load(self):
        indices = sorted(self.file.iterIndices())
        n = self.nextIndex = max(indices) + 1
        for name, _ in dict.items(self.lists):
            self.lists[name] = [None] * self.nextIndex
        
        for i in indices:
            for name in self.lists:
                self.op[name].load(i)
                
        if self.key is not None:
            self.assignKey(self.key, self.keyFunc)
            
    def save(self, **kwargs, keep=True):
        assert self.key in kwargs, "call to `save` must include a kwarg of the key name"
        keyVal = self.keyFunc(kwargs[self.key], client=True)
        
        if keyVal not in self.tbl:
            for name in self.lists:
                assert len(self.lists[name]) == self.nextIndex, "Internal error"
                self.lists[name].append(None)
            self.tbl[keyVal] = self.nextIndex
            self.nextIndex += 1
        
        i = self.tbl[keyVal]
        
        for name in kwargs:
            if name not in self.op:
                raise AssertionError(f"given save item '{name}' does not have data store ops initialized")
            self.op[name].save(kwargs[name], keep=keep)
            
    def get(self, name, **kwargs):
        assert len(kwargs) == 1 and self.key in kwargs, "must give assigned 'key' to get corresponing data"
        keyVal = self.keyFunc(kwargs[self.key], client=True)
        if keyVal not in self.tbl:
            return None
        i = self.tbl[keyVal]
        return self.lists[name][i]
        
    ##################### INSTANCE BUILDING METHODS ##################### : 
    def assignKey(self, name, func):
        assert name is not None, "name to be used as key cannot be None"
        self.key = name
        self.keyFunc = func
        self.tbl = {self.keyFunc(x): i for i,x in enumerate(self.lists[self.key]) if x is not None}
        
    def assignOperations(self, name, *, load, save, keep, keepClient):
        class the(O()):
            def load(*, i, keep=True):           
                x = load(self.file.io[name].read(i=i)) if load else None
                if self.op[name].keep:
                    self.lists[name][i] = self.op[name].keep(x)
                return x
            
            def save(x, *, i, keep=True):
                self.file.io[name].write(save(x)) if save else None
                if self.op[name].keepClient:
                    self.lists[name][i] = self.op[name].keepClient(x)
                
            def keep(x, *, i, ):
                kept = None
                if keep:
                    kept = keep(x)
                    self.lists[name][i] = kept
                return kept
                
            def keepClient(x, *, i, ):
                kept = None
                if keepClient:
                    kept = keepClient(x)
                    self.lists[name][i] = kept
                return kept
            
        self.op[name] = the
        if keep and name not in self.lists:
            self.lists[name] = [None] * self.nextIndex
    
    
@staticmethod
def __IndexDataStore__from_specs(specs, **kwargs):
    for k, v in kwargs.items():
        specs[k] = v
    f = IndexFileSystem(specs.dir, specs.key)
    d = IndexDataStore(f)
    d.assignKey(specs.key, specs.keyFunc)
    for name, val in dict.items(specs):
        f.assignIO(name, read=val.read, write=val.write, format=val.format)
        d.assignOperations(name, load=val.load, save=val.save, keep=val.keep, keepClient=val.keepClient)
    d.load()
IndexDataStore.from_specs = __IndexDataStore__from_specs

In [ ]:
class PDS(O()):
    '''data transformation code inside `op` object:

    client --save(+write)--> disk; client --keep--> memory; disk --load(+read)--> memory

    read = text/bytes stream -> as-is object read from file
    write = object to save as is to file -> text/bytes stream
    load = as-is object read from file -> object to be loaded in memory
    save = raw object given by client -> object to save as-is to file
    keep = as-is object read from file -> object to keep in memory
    keepClient = raw object given by client -> object to keep in memory
    '''

    key = 'Params'
    keyFunc = lambda params, *, client=False: tuple(sorted(dict.items(params)))

    class op(O()):
        class Params(O()):
            format = 'text'
            read = json.loads
            write = json.dumps
            load = I
            save = I
            keep = I
            keepClient = dict

        class Results(O()):
            format = 'bytes'
            read = pickle.loads
            write = pickle.dumps
            load = TODO
            save = TODO
            keep = TODO
            keepClient = 

        class Extras(O()):
            format = 'bytes'
            read = pickle.loads
            write = pickle.dumps
            load = TODO
            save = TODO
            keep = False

In [129]:
class FDS(O()):
    key = 'Feat'
    keyFunc = lambda features
    TODO

class ModelManager():
    def __init__(self, directory):
        self.dir = directory if isinstance(directory, Path) else Path(directory)
        self.F = IndexDataStore.from_specs(FDS, dir=self.dir/'.features')
        self.S = IndexDataStore.from_specs(SDS, dir=self.dir/'.samples')
    
    def make_param_manager(self, features, samples):
        fset = features if isinstance(features, frozenset) else frozenset(features)
        if fset not in self.fTbl: 
            self.init_features(fset)
        stup = samples; assert isinstance(stup, tuple) and len(stup==3)
        if stup not in self.sTbl:
            self.init_samples(stup)
        fs_ids = (self.fTbl[fset], self.sTbl[stup])
        if fs_ids not in self.fsSet:
            self.init_features_samples_with_ids(*fs_ids)
        return ParamManager()
        
    def init_features(self, fset):
        TODO
    
    def init_samples(self, stup):
        TODO
        
    def init_features_samples_with_ids(fId, sId):
        TODO

IndentationError: expected an indented block (<ipython-input-129-50f47f83cf9e>, line 3)

## old shit / testing

In [105]:
# PUREL TESTING
class HI(O()):
    class DERP():
        val = -99
        dir = Path('.')
        def yo(x):
            def y(a):
                return x(a) + 1000
            return y
        my = O(what=yo(lambda a: a ** 2))
        hm = O(func=lambda x: x * __class__.val, goto=lambda x: __class__.dir/x)
        class WHAT(O()):
            dude = lambda x: x + 10
        class YES(O()):
            dude = WHAT.dude
HI.DERP.my.what(5)
HI.DERP.hm.func(5)
HI.DERP.dir = Path('what')
HI.DERP.hm.goto('.git')
HI.DERP.WHAT.dude(0)

# ANOTHER PURE TESTING
def dothe(self):
    self.val = 4
    top = 100
    class Derp(O()):
        def func(x):
            return x + self.val
        def top(y):
            return y * top
    self.derp = Derp
item = Stop()
dothe(item)
item.derp.top(3)

class Yo(O()):
    class What(O()):
        derp = 5
    class Another(O()):
        yep = What.derp
Yo()

NameError: name 'What' is not defined

In [ ]:
class ParamManager():
    
    @staticmethod
    def paramsKey(params, *, client=False):
        '''return key that identifies params, regardless of whether params is internal on client given'''
        return tuple(sorted(dict.items(params)))
    
    
    class file(metaclass=staticclass):
        dir = None # don't forget to attach me in the code a level above!
        
        def iterIndices():
            for dot_params in __class__.dir.glob('*.params'):
                yield int(dot_params.stem)
        
        def getFilePath(name, *, i):
            return __class__.dir / (str(i) + name)
        
        def readWrapper(read):
            '''wraps io read operations to safely return None if file does not exist'''
            @wraps(read)
            def read_safely(*a, **k):
                try:
                    return read(*a, **k)
                except FileNotFoundError:
                    return None
            return read_safely
        
        io = O(
            Params = O(
                read = readWrapper( lambda *,i: json.loads(__class__.getFilePath('.params', i=i).read_text()) ),
                write = lambda x,*,i: __class__.getFilePath('.params', i=i).write_text(json.dumps(x))
            ),
            Results = O(
                read = readWrapper( lambda *,i: pickle.load(__class__.getFilePath('.results', i=i).read_bytes()) ),
                write = lambda x,*,i: __class__.getFilePath('.results', i=i).write_bytes( pickle.dumps(x) )
            ),
            Extras = O(
                read = readWrapper( lambda *,i: pickle.load(__class__.getFilePath('.extras', i=i).read_bytes()) ),
                write = lambda x,*,i: __class__.getFilePath('.extras', i=i).write_bytes( pickle.dumps(x) )
            ),
        )
            
    class opt(O()):
        '''data transformation code:
        
        client --save--> disk; client --keep--> memory; disk --load--> memory
        
        load = object raw read from file -> object to be loaded in memory
        save = raw object given by client -> object to save to file
        keep = raw object given by client -> object to keep in memory
        '''
        class Params(metaclass=staticclass):
            load = I
            save = I
            keep = I
            
        class Results(metaclass=staticclass):
            load = TODO
            save = TODO
            keep = TODO
            
        class Extras(metaclass=staticclass):
            load = TODO
            save = TODO
            keep = TODO
            
    
    def __init__(self, directory):
        self.file.dir = directory if isinstance(directory, Path) else Path(directory)
        self.opt = O.mycopy(__class__.opt)
        self.reload_runs()
        
        
    def reload_runs(self):
        indices = sorted(self.file.iterIndices())
        m = max(indices) + 1
        self.paramsList = [None] * m
        self.resultsList = [None]*m if self.opt.Results.load else None
        self.extrasList = [None]*m if self.opt.Extras.load else None
        
        for i in indices:
            paramsRead = self.file.io.Params.read(i=i)
            self.paramsList[i] = self.opt.Params.load(paramsRead)
            
            if self.opt.Results.load:
                resultsRead = self.file.io.Results.read(i=i)
                self.resultsList[i] = self.opt.Results.load(resultsRead)
            
            if self.opt.Extras.load:
                extrasRead = self.file.io.Extras.read(i=i)
                self.extrasList[i] = self.opt.Extras.load(extrasRead)
                
        self.paramsTbl = {self.paramsKey(p): i for i,p in enumerate(self.paramsList) if p is not None}
        
        
    def save_run(params, results, extras):
        paramsClient, resultsClient, extrasClient = params, results, extras; del params, results, extras
        params = self.opt.Params.keep(paramsClient)
        key = self.paramsKey(params)
        if key not in self.paramsTbl:
            self.paramsList.append(params)
            self.paramsTbl[key] = len(self.paramsList)-1
        
        i = self.paramsTbl[key]
        
        # save everything to file, overwriting if things were there before
        
        paramsWrite = self.opt.Params.save(paramsClient)
        self.file.io.Params.write(paramsWrite, i=i)
                
        if self.opt.Results.save:
            resultsWrite = self.opt.Results.save(resultsClient)
            self.file.io.Results.write(resultsWrite, i=i)
            
        if self.opt.Extras.save:
            extrasWrite = self.opt.Extras.save(extrasClient)
            self.file.io.Extras.write(extrasWrite, i=i)
            
        # save internal version of client data to memory
        
        if self.opt.Results.keep:
            if self.resultsList is None:
                self.resultsList = []
            if i >= len(self.resultsList):
                self.resultsList.extend([None] * (i+1 - len(self.resultsList)))
            self.resultsList[i] = self.opt.Results.keep(resultsClient)
        
        if self.opt.Extras.keep:
            if self.extrasList is None:
                self.extrasList = []
            if i >= len(self.extrasList):
                self.extrasList.extend([None] * (i+1 - len(self.extrasList)))
            self.extrasList[i] = self.opt.Extras.keep(extrasClient)
        
        
    def get_results(params):
        key = self.paramsKey(params, client=True)
        if key not in self.paramsTbl:
            return None
        i = self.paramsTbl[key]
        if i >= len(self.resultsList) or self.resultsList[i] is None:
            return None
        return self.resultsList[i]
        
        
    def get_extras(params):
        key = self.paramsKey(params, client=True)
        if key not in self.paramsTbl:
            return None
        i = self.paramsTbl[key]
        if i >= len(self.extrasList) or self.extrasList[i] is None:
            return None
        return self.extrasList[i]
        
        

In [ ]:
class ModelManager():
    def __init__(self, directory):
        self.dir = directory
        self.next_group_index = ?
        self.next_s
    
    def make_param_manager(self, features, samples):
        fset = features if isinstance(features, frozenset) else frozenset(features)
        if fset not in self.fTbl: 
            self.init_features(fset)
        stup = samples; assert isinstance(stup, tuple) and len(stup==3)
        if stup not in self.sTbl:
            self.init_samples(stup)
        fs_ids = (self.fTbl[fset], self.sTbl[stup])
        if fs_ids not in self.fsSet:
            self.init_features_samples_with_ids(*fs_ids)
        return ParamManager()
        
    def init_features(self, fset):
        TODO
    
    def init_samples(self, stup):
        TODO
        
    def init_features_samples_with_ids(fId, sId):
        TODO

In [ ]:
# OLD

class ModelManager():
    def __init__(self, directory):
        self.dir = directory
        self.next_group_index = ?
        self.next_s
    
    def make_param_manager(self, features, samples):
        fset = features if isinstance(features, frozenset) else frozenset(features)
        if fset not in self.fTbl: 
            self.init_features(fset)
        stup = samples; assert isinstance(stup, tuple) and len(stup==3)
        if stup not in self.sTbl:
            self.init_samples(stup)
        fs_ids = (self.fTbl[fset], self.sTbl[stup])
        if fs_ids not in self.fsSet:
            self.init_features_samples_with_ids(*fs_ids)
        return ParamManager()
        
    def init_features(self, fset):
        TODO
    
    def init_samples(self, stup):
        TODO
        
    def init_features_samples_with_ids(fId, sId):
        TODO

In [127]:
s = ParamSearch(PSD)
search = s.search()
cnt = 0
for i,p in zip(range(999),search):
    print(p)
    search.send(i)
    cnt += 1
print(cnt)

{'max_depth': 6, 'num_leaves': 64, 'min_data_in_leaf': 60, 'min_sum_hessian_in_leaf': 50, 'lambda_l1': 0, 'lambda_l2': 0}
{'max_depth': 6, 'num_leaves': 64, 'min_data_in_leaf': 10, 'min_sum_hessian_in_leaf': 50, 'lambda_l1': 0, 'lambda_l2': 0}
{'max_depth': 6, 'num_leaves': 64, 'min_data_in_leaf': 150, 'min_sum_hessian_in_leaf': 50, 'lambda_l1': 0, 'lambda_l2': 0}
{'max_depth': 6, 'num_leaves': 64, 'min_data_in_leaf': 60, 'min_sum_hessian_in_leaf': 6.1, 'lambda_l1': 0, 'lambda_l2': 0}
{'max_depth': 6, 'num_leaves': 64, 'min_data_in_leaf': 60, 'min_sum_hessian_in_leaf': 100.0, 'lambda_l1': 0, 'lambda_l2': 0}
{'max_depth': 6, 'num_leaves': 64, 'min_data_in_leaf': 60, 'min_sum_hessian_in_leaf': 50, 'lambda_l1': 0.01, 'lambda_l2': 0}
{'max_depth': 6, 'num_leaves': 64, 'min_data_in_leaf': 60, 'min_sum_hessian_in_leaf': 50, 'lambda_l1': 0.11, 'lambda_l2': 0}
{'max_depth': 6, 'num_leaves': 64, 'min_data_in_leaf': 60, 'min_sum_hessian_in_leaf': 50, 'lambda_l1': 0, 'lambda_l2': 0.01}
{'max_dept

# SCRATCH TESTING

In [128]:
def hi():
    for i in range(10):
        first = yield i
        assert (yield)==None
        print(' here', first)

In [129]:
h = hi()
for x in h:
    print('x', x)
    h.send(x / 10.)
    #print(next(h))

x 0
 here 0.0
x 1
 here 0.1
x 2
 here 0.2
x 3
 here 0.3
x 4
 here 0.4
x 5
 here 0.5
x 6
 here 0.6
x 7
 here 0.7
x 8
 here 0.8
x 9
 here 0.9
